In [1]:
import numpy as np
import scipy.sparse as spsp
from scipy.sparse import csc_matrix, linalg as spspla


rows,cols=np.loadtxt("./p2p-Gnutella30/p2p-Gnutella30.mtx", unpack = True) #,skiprows=83,unpack=True)
rows=rows-1; cols=cols-1;


In [2]:
print(len(rows))
print(rows)

88328
[ 1311.  1629.  5352. ... 36617. 36617. 36641.]


In [3]:
data = np.repeat(1, len(rows))
print(len(data))
print(data)

88328
[1 1 1 ... 1 1 1]


In [4]:
A=spsp.coo_matrix((data,(rows,cols)),shape=(36682,36682))
print(A.shape)

(36682, 36682)


In [5]:
print(A)

  (1311, 0)	1
  (1629, 0)	1
  (5352, 0)	1
  (9391, 0)	1
  (0, 1)	1
  (3050, 1)	1
  (16254, 1)	1
  (21629, 1)	1
  (26775, 1)	1
  (28911, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1331, 3)	1
  (3675, 3)	1
  (5014, 3)	1
  (11048, 3)	1
  (15239, 3)	1
  (18724, 3)	1
  (30694, 3)	1
  (31271, 3)	1
  (31803, 3)	1
  (34383, 3)	1
  (0, 4)	1
  (1163, 4)	1
  (3500, 4)	1
  :	:
  (36649, 36657)	1
  (36649, 36658)	1
  (36619, 36659)	1
  (36619, 36660)	1
  (36619, 36661)	1
  (36650, 36662)	1
  (36650, 36663)	1
  (36650, 36664)	1
  (36658, 36665)	1
  (36658, 36666)	1
  (14860, 36667)	1
  (36088, 36668)	1
  (36088, 36669)	1
  (36628, 36670)	1
  (36628, 36671)	1
  (36655, 36672)	1
  (36655, 36673)	1
  (36655, 36674)	1
  (36674, 36675)	1
  (36674, 36676)	1
  (36674, 36677)	1
  (36617, 36678)	1
  (36617, 36679)	1
  (36617, 36680)	1
  (36641, 36681)	1


In [6]:
A

<36682x36682 sparse matrix of type '<class 'numpy.int64'>'
	with 88328 stored elements in COOrdinate format>

In [7]:
cscA = spsp.csc_matrix(A)


In [8]:
cscA

<36682x36682 sparse matrix of type '<class 'numpy.int64'>'
	with 88328 stored elements in Compressed Sparse Column format>

In [9]:
print(cscA)

  (1311, 0)	1
  (1629, 0)	1
  (5352, 0)	1
  (9391, 0)	1
  (0, 1)	1
  (3050, 1)	1
  (16254, 1)	1
  (21629, 1)	1
  (26775, 1)	1
  (28911, 1)	1
  (0, 2)	1
  (0, 3)	1
  (1331, 3)	1
  (3675, 3)	1
  (5014, 3)	1
  (11048, 3)	1
  (15239, 3)	1
  (18724, 3)	1
  (30694, 3)	1
  (31271, 3)	1
  (31803, 3)	1
  (34383, 3)	1
  (0, 4)	1
  (1163, 4)	1
  (3500, 4)	1
  :	:
  (36649, 36657)	1
  (36649, 36658)	1
  (36619, 36659)	1
  (36619, 36660)	1
  (36619, 36661)	1
  (36650, 36662)	1
  (36650, 36663)	1
  (36650, 36664)	1
  (36658, 36665)	1
  (36658, 36666)	1
  (14860, 36667)	1
  (36088, 36668)	1
  (36088, 36669)	1
  (36628, 36670)	1
  (36628, 36671)	1
  (36655, 36672)	1
  (36655, 36673)	1
  (36655, 36674)	1
  (36674, 36675)	1
  (36674, 36676)	1
  (36674, 36677)	1
  (36617, 36678)	1
  (36617, 36679)	1
  (36617, 36680)	1
  (36641, 36681)	1


In [10]:
print(cscA[0,0])
print(cscA[1311, 0])

0
1


We already have uploaded the corresponding binary matrix: $A=$ `cscA` $ \in \mathbb{R}^{n\times n}$ where in this case $n =36682$.

Consider then the fixed point problem $Ax = x$. Remember that if the web network does not contain dangling nodes, then the matrix $A$ is column stochastic, i.e. $1 \in Spec(A)$. If unique, the eigenvector of eigenvalue 1 is the so-called **PR vector**.

However for disconnected networks the PR vector is not unique. On the other hand, if the network has dangling nodes then the matrix $A$ is column substochastic (and has no eigenvector of eigenvalue 1). In order to adress those two problems we may consider:
$$
M_m = (1-m) A + mS
$$
where $0 \leq m \leq 1$ is a damping factor, which we may consider to be fixed at $m = 0.15$, and $mS = e z^t$, where $e = ( 1 , \dots, 1)^T$ and $z= (z_1, \dots, z_n)^T$ is the vector given by $z_j = m/n$ if the column $j$ of the matrix $A$ contains non-zero elements (this is the node $j$ has a link to any other node) and $z_j = 1/n$ otherwise. 

Let $G=(g_{ij})$ be the link matrix, that is $g_{ij} =1$ when there is a link between the pages $i, j$ and $g_{ij}=0$ if there is no link. Then, $n_j = \sum_i g_{ij}$ is the out-degree of the page j. Let now $D=diag(d_{11}, \dots, d_{nn})$, with $d_{jj} = 1/n_j$ if $n_j \neq 0$ and $d_{jj} = 0$ otherwise. Then $A = GD$

Now, if we seek to compute the PR vector of $M_m$ using the power method, we may consider the algorithm given by the iteration of 
$$
x_{k+1} = (1- m ) GD x_k + e z^t x_k
$$
until $\| x_{k+1} - x_k \| < tol$. 

In [11]:
m = 0.15
n = cscA.shape[0]
## we compute d_i in order to calculate D and z_j in order to compute z
z = np.zeros((cscA.shape[0],1))
d = np.zeros((cscA.shape[0],1))
n_j = np.zeros((cscA.shape[0],1))

for j in range(cscA.shape[1]):
    n_j[j] = np.sum(cscA[:, j])
    if n_j[j] > 0:
        d[j] = 1/n_j[j]
        z[j] = m/n
    else:
        d[j] = 0
        z[j] = 1/n
        
print(d)
print(n_j)
print(z)

[[0.25      ]
 [0.16666667]
 [1.        ]
 ...
 [1.        ]
 [1.        ]
 [1.        ]]
[[4.]
 [6.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]
[[4.08919906e-06]
 [4.08919906e-06]
 [4.08919906e-06]
 ...
 [4.08919906e-06]
 [4.08919906e-06]
 [4.08919906e-06]]


We treat the given matrix as the link matrix $G$, since it is a binary matrix, indicating whether there exists a link between nodes. Then we have the vectors $e, z, d$ as computed previously, and we can then define the matrix $D$, and the step of the algorithm:

In [12]:
print(d.shape)
D = np.zeros((d.shape[0],d.shape[0]))
np.fill_diagonal(D, d)
print(D.shape)
print(D)

(36682, 1)
(36682, 36682)
[[0.25       0.         0.         ... 0.         0.         0.        ]
 [0.         0.16666667 0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In order to compute the multiplication of the sparse matrix and the vector $x_0$ we have to create a function that conducts the matrix product. Note that every nonzero coefficient will be a 1, therefore, the nonzero coefficients are indicatin us which coefficients of the vector $x_0$ we should be adding up. 

In [13]:
x0 = np.ones((n, 1))
dx = (d.T*x0).T
print(dx.shape)        

KeyboardInterrupt: 

In [14]:
dxx = d*x0

In [15]:
print(dxx.shape)
print(dxx)

(36682, 1)
[[0.25      ]
 [0.16666667]
 [1.        ]
 ...
 [1.        ]
 [1.        ]
 [1.        ]]


In [14]:
print(rows)
print(cols)
print(data)
x0 = np.repeat(1, d.shape[0])
y = np.zeros((x0.shape))

def multSparse(rows, cols, x0):
    y = np.zeros((x0.shape))
## We seek to compute A*x
    for i in range(len(rows)):
        row = int(rows[i])
        col = int(cols[i])
        #print(row, col)
        y[row] +=  x0[col] 
        ## A[rows[i], cols[i]]= 1 always, then this is equivalent to y[row] + A[row, col] * x0[col]
    print(y)
    return y

multSparse(rows, cols, x0)

[ 1311.  1629.  5352. ... 36617. 36617. 36641.]
[    0.     0.     0. ... 36679. 36680. 36681.]
[1 1 1 ... 1 1 1]
[10.  0. 10. ...  0.  0.  0.]


array([10.,  0., 10., ...,  0.,  0.,  0.])

In [3]:
import numpy as np
x0 = np.ones((30,1))
print(x0.shape)

(30, 1)


In [ ]:
tol = 1e-16
iter_max = 1e3
#x0 = np.repeat(1, d.shape[0])
x0 = np.ones((n, 1))
xk = np.zeros((n, 1))
for k in range(int(iter_max)):
    print('---------------')
    print('Iteration:', k)
    
    zx = np.matmul(z.T, x0)
    ezx = np.repeat(zx, x0.shape[0])
    dx = (d.T*x0).T
    print(dx.shape)
    print(cscA.shape)
    
    GDx = multSparse(rows,cols, dx)
    
    for i in range(GDx.shape[0]):
        xk[i] = (1-m) * GDx[i] + zx[0]
    #xk = (1- m) * multSparse(rows,cols, dx) + ezx
    #xk = (1- m) * np.matmul(cscA, dx) + ezx
    print(xk.shape)
    
    print(xk, x0)
    print(np.linalg.norm(xk - x0))
        
    if np.linalg.norm(xk - x0) < tol:
        break
    else: 
        x0 = xk


---------------
Iteration: 0


In [ ]:
## It takes a lot to compute the norm between xk and x0 ?? 
## we could create a function that computes it actually

In [33]:
## first iteration
tol = 1e-16
iter_max = 1e3
x0 = np.repeat(1, d.shape[0])
print(x0.shape)
x0.reshape((n,1))
print(x0.shape)
print(x0, z, d)
print(m/n)
print(1/n)
print(max(z))
zx = np.matmul(z.T, x0)
ezx = np.repeat(zx, x0.shape[0])
dx = (d.T*x0).T
GDx = multSparse(rows,cols, dx)
print(zx, ezx)
print('dx:', dx)
print('GDx:', GDx)

(36682,)
(36682,)
[1 1 1 ... 1 1 1] [[4.08919906e-06]
 [4.08919906e-06]
 [4.08919906e-06]
 ...
 [4.08919906e-06]
 [4.08919906e-06]
 [4.08919906e-06]] [[0.25      ]
 [0.16666667]
 [1.        ]
 ...
 [1.        ]
 [1.        ]
 [1.        ]]
4.089199062210348e-06
2.7261327081402323e-05
[2.72613271e-05]
[[5.13852814]
 [0.        ]
 [4.74622132]
 ...
 [0.        ]
 [0.        ]
 [0.        ]]
[0.15530642] [0.15530642 0.15530642 0.15530642 ... 0.15530642 0.15530642 0.15530642]
dx: [[0.25      ]
 [0.16666667]
 [1.        ]
 ...
 [1.        ]
 [1.        ]
 [1.        ]]
GDx: [[5.13852814]
 [0.        ]
 [4.74622132]
 ...
 [0.        ]
 [0.        ]
 [0.        ]]


In [34]:
nonzro = []

for i in range(GDx.shape[0]):
    if GDx[i] > 0 :
        nonzro.append(i)
        
#print(nonzro)
print(len(nonzro))

#norma = np.linalg.norm(x0 - GDx)
#print(norma)
xk = GDx + ezx
print(xk) 
print(np.linalg.norm(x0 - xk))

9722
[[5.29383456 5.29383456 5.29383456 ... 5.29383456 5.29383456 5.29383456]
 [0.15530642 0.15530642 0.15530642 ... 0.15530642 0.15530642 0.15530642]
 [4.90152774 4.90152774 4.90152774 ... 4.90152774 4.90152774 4.90152774]
 ...
 [0.15530642 0.15530642 0.15530642 ... 0.15530642 0.15530642 0.15530642]
 [0.15530642 0.15530642 0.15530642 ... 0.15530642 0.15530642 0.15530642]
 [0.15530642 0.15530642 0.15530642 ... 0.15530642 0.15530642 0.15530642]]


KeyboardInterrupt: 

In [21]:
print(GDx.shape, GDx)
print(ezx.shape, ezx)

(36682, 1) [[5.13852814]
 [0.        ]
 [4.74622132]
 ...
 [0.        ]
 [0.        ]
 [0.        ]]
(36682,) [0.15530642 0.15530642 0.15530642 ... 0.15530642 0.15530642 0.15530642]


In [30]:
xk = np.zeros((GDx.shape[0],1))
for i in range(GDx.shape[0]):
    xk[i] = GDx[i] + zx[0]
    
print(xk.shape, xk)

(36682, 1) [[5.29383456]
 [0.15530642]
 [4.90152774]
 ...
 [0.15530642]
 [0.15530642]
 [0.15530642]]


In [29]:
print(zx.shape, zx, zx[0])
print(GDx[0] + zx[0])
print(GDx[0])

(1,) [0.15530642] 0.15530641731639283
[5.29383456]
[5.13852814]


In [ ]:
m = 0.15
n = cscA.shape[0]
## we compute d_i in order to calculate D and z_j in order to compute z
z = np.zeros((cscA.shape[0],1))
d = np.zeros((cscA.shape[0],1))
n_j = np.zeros((cscA.shape[0],1))

for j in range(cscA.shape[1]):
    suma = 0
    for i in range(cscA.shape[0]):
        if cscA[i,j] !=0:
            suma += 1
    n_j[j] = suma
    if n_j[j] > 0:
        d[j] = 1/n_j[j]
        z[j] = m/n
    else:
        d[j] = 0
        z[j] = 1/n
        
print(d)
print(n_j)
print(z)

In [16]:
z = np.zeros(( cscA.shape[0], 1))
print(z.shape)

(1, 36682)


**EX2: Compute the PR vector of Mm using the power method without storing matrices.**

1. From the vectors that store the link matrix G obtain, for each j = 1, ..., n, the set of indices Lj corresponding to pages having a link with page j.

In [26]:
print(np.max(n_j))
print(n_j[0])
print(n_j[1])
print(cscA[:,0])
print(cscA.indices)
print(len(cscA.indices))
print(cscA.indptr)
## the indices pointer array is an array in this case containing the sum of
## the number of indices corresponding to the first column, second, ...

## i.e. it indicates when we are skipping col, 
## 

54.0
[4.]
[6.]
  (1311, 0)	1
  (1629, 0)	1
  (5352, 0)	1
  (9391, 0)	1
[ 1311  1629  5352 ... 36617 36617 36641]
88328
[    0     4    10 ... 88326 88327 88328]


When expressing the sparse matrix in terms of the index array and pointer index array, the indices pointer array is an array in this case containing the sum of the number of indices corresponding to the first column, second column and so on. This is it indicates when we are skipping col. More formally, the pointer array has as coefficient j the number of elements until column j, i.e. $\sum_{i=1}^j n_i$. 

We want the array with the $n_j$ indexes containing links, so for each column, this is for each k in the pointer index vector, we have $n_k$ indexes in the index vector that correspond to the pages which have links with page k:

In [31]:
indexs = cscA.indices
point_ind = cscA.indptr

Lj = np.zeros((n, int(np.max(n_j))))
for j in range(n):
    for k in range(int(n_j[j])):
        Lj[j, k] = indexs[k + point_ind[j]]
    
print(Lj[0])


[1311. 1629. 5352. 9391.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.]


In [40]:
indexs = cscA.indices
point_ind = cscA.indptr
nj = np.zeros((n,1))
Lj = np.zeros((n,54))
for j in range(n):
    nj[j] = point_ind[j+1] - point_ind[j]
    for k in range(int(nj[j])):
        Lj[j, k] = indexs[k + point_ind[j]]
    
print(Lj[0])
print(nj)

[1311. 1629. 5352. 9391.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.    0.    0.    0.]
[[4.]
 [6.]
 [1.]
 ...
 [1.]
 [1.]
 [1.]]


In [49]:
for i in Lj[0,:int(nj[0])]:
    print(i)

1311.0
1629.0
5352.0
9391.0


In [44]:
print(len(point_ind))
print(point_ind[3])
print(point_ind)

36683
11
[    0     4    10 ... 88326 88327 88328]


2. Each row of $L_j$ has $n_j$ elements. So we can get the indexs we need for each $j$, this is, for each column of the link matrix. 

3. Iterate $x_{k+1} = M_mx_k$ until $\|x_{k+1} − x_k\|_{\infty} < tol$.

We do not want to compute explicitely the matrix $M_m$. Note that if $n_j = 0$, then we have $M_{ij} = 1/n$ for all $1 \leq i \leq n$. Then, recalling that $g_{ij} = 0$ iff $i \notin L_j$, then the product $Mx$ can be implemented as follows:

In [33]:
xc=x
x=0
for j in range (0,n):
    if(n[j]==0):
        x=x+xc[j]/n
    else:
        for i in L[j]:
            x[i]=x[i]+xc[j]/n[j]
x=(1-m)*x+m/n


[    0.  3050. 16254. 21629. 26775. 28911.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
     0.     0.     0.     0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
